In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Dispositivo disponible: {device}")

Dispositivo disponible: cpu


In [2]:
from torch.utils.data import Dataset
class MultilabelDataset(Dataset):
  def __init__(self, num_images=20, transform=None):
    self.num_images = num_images
    self.transform = transform
    self.images = torch.randn(num_images, 3, 28, 28)
    self.labels = torch.randint(0, 2, (num_images, 3))

  def __len__(self):
      return self.num_images

  def __getitem__(self, idx):
      image = self.images[idx]
      label = self.labels[idx]
      if self.transform:
          image = self.transform(image)
      return image, label

In [3]:
from torchvision import transforms

transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
from torch.utils.data import DataLoader

dataset = MultilabelDataset(transform=transformations)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [5]:
from torchvision.models import alexnet
import torch.nn as nn

model = alexnet(pretrained=True)

for param in model.features.parameters():
    param.requires_grad = False

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 3)

model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)

In [7]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device).float()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

Epoch 1, Loss: 0.8132599194844564
Epoch 2, Loss: 0.7533950209617615
Epoch 3, Loss: 0.7761632402737936
Epoch 4, Loss: 0.6954896450042725
Epoch 5, Loss: 0.6826885143915812


In [11]:
from sklearn.metrics import classification_report

model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in dataloader: # En la práctica aquí se deberan usar datos de test
        inputs = inputs.to(device)
        labels = labels.numpy(force=True)

        outputs = model(inputs)
        preds = torch.sigmoid(outputs).numpy(force=True) > 0.5

        all_labels.extend(labels)
        all_preds.extend(preds)

print(classification_report(all_labels, all_preds, target_names=["Clase_1", "Clase_2", "Clase_3"]))

              precision    recall  f1-score   support

     Clase_1       0.65      1.00      0.79        13
     Clase_2       0.50      1.00      0.67        10
     Clase_3       0.82      0.82      0.82        11

   micro avg       0.63      0.94      0.75        34
   macro avg       0.66      0.94      0.76        34
weighted avg       0.66      0.94      0.76        34
 samples avg       0.63      0.81      0.69        34



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
